In [1]:
#import Libraries
import psycopg2
import numpy as np
import pandas as pd
pd.options.display.max_columns = 300

In [2]:
#connect SQL
conn = psycopg2.connect(database='usaspendingdb', user='postgres', password='Fatih123', host='127.0.0.1', port='5432')

In [3]:
sql_cols = ('federal_action_obligation, '
            'base_and_exercised_options_value, '
            'base_and_all_options_value, '
            'awarding_sub_agency_name, '
            'awarding_office_name, '
            'funding_sub_agency_name, '
            'funding_office_name, '
            'primary_place_of_performance_state_code, '
            'award_or_idv_flag, '
            'award_type, '
            'type_of_contract_pricing, '
            'dod_claimant_program_description, '
            'type_of_set_aside_code, '
            'contract_bundling, '
            'national_interest_action, '
            'gfe_gfp, '
            'contract_financing, '
            'portfolio_group, '
            'product_or_service_code_description, '
            'naics_bucket_title, '
            'naics_description'
            )

In [4]:
#Create DF
df = pd.read_sql_query('SELECT ' + sql_cols + ' FROM consolidated_data_filtered_bucketed', con=conn)
df.shape

(35414, 21)

In [5]:
#Check if there is any null in DF.
df.isnull().sum()

federal_action_obligation                     0
base_and_exercised_options_value              0
base_and_all_options_value                    0
awarding_sub_agency_name                      0
awarding_office_name                          0
funding_sub_agency_name                       0
funding_office_name                          43
primary_place_of_performance_state_code       0
award_or_idv_flag                             0
award_type                                    0
type_of_contract_pricing                      0
dod_claimant_program_description              1
type_of_set_aside_code                      414
contract_bundling                             0
national_interest_action                      0
gfe_gfp                                       0
contract_financing                            0
portfolio_group                               0
product_or_service_code_description           0
naics_bucket_title                         7870
naics_description                       

In [6]:
#Drop null rows from 'type_of_set_aside_code' column.
df = df[pd.notnull(df['type_of_set_aside_code'])]
df.shape

(35000, 21)

In [7]:
df['type_of_set_aside_code'].values

array(['NONE', 'NONE', 'NONE', ..., 'NONE', 'SBA', 'NONE'], dtype=object)

In [8]:
df['type_of_set_aside_code'].value_counts()

NONE       22154
SBA         8619
WOSB        1379
8A          1222
8AN         1185
SDVOSBC      271
HZC          108
SBP           30
SDVOSBS       14
EDWOSB         7
WOSBSS         6
HZS            4
ISBEE          1
Name: type_of_set_aside_code, dtype: int64

In [9]:
df['set_aside_number'] = df['type_of_set_aside_code'].map({'NONE':1, 'SBA':2, 'WOSB':3, '8A':4, '8AN':5, 'SDVOSBC':6,'HZC':7,
                                                           'SBP':8, 'SDVOSBS':9, 'EDWOSB':10, 'WOSBSS':11, 'HZS':12, 'ISBEE':13})

In [10]:
df['set_aside_number'].value_counts()

1     22154
2      8619
3      1379
4      1222
5      1185
6       271
7       108
8        30
9        14
10        7
11        6
12        4
13        1
Name: set_aside_number, dtype: int64

In [11]:
def contract_value(c):
    if c['base_and_exercised_options_value'] > 0:
        return c['base_and_exercised_options_value']
    elif c['base_and_all_options_value'] > 0:
        return c['base_and_all_options_value']
    elif c['federal_action_obligation'] > 0:
        return c['federal_action_obligation'] 
    else:
        return 0

In [12]:
df['contract_value'] = df.apply(contract_value, axis=1)

In [13]:
#Drop columns that we dont need anymore.
df = df.drop(['type_of_set_aside_code','base_and_exercised_options_value','base_and_all_options_value',
             'federal_action_obligation'], axis=1)

In [14]:
#Create another DF with non null columns and rows.
df2 = df.dropna()
df2.shape

(27246, 19)

In [15]:
df2.head()

,awarding_sub_agency_name,awarding_office_name,funding_sub_agency_name,funding_office_name,primary_place_of_performance_state_code,award_or_idv_flag,award_type,type_of_contract_pricing,dod_claimant_program_description,contract_bundling,national_interest_action,gfe_gfp,contract_financing,portfolio_group,product_or_service_code_description,naics_bucket_title,naics_description,set_aside_number,contract_value
0,USTRANSCOM,USTRANSCOM-AQ,DEPT OF THE AIR FORCE,HQ AMC TE,TX,AWARD,DO,FIRM FIXED PRICE,SERVICES,NOT A BUNDLED REQUIREMENT,NONE,TRANSACTION DOES NOT USE GFE/GFP,NOT APPLICABLE,Transportation Services,TRANSPORTATION/TRAVEL/RELOCATION- TRAVEL/LODGI...,Air TransportationT,NONSCHEDULED CHARTERED PASSENGER AIR TRANSPORT...,1,64925.82
1,WASHINGTON HEADQUARTERS SERVICES (WHS),WASHINGTON HEADQUARTERS SERVICES,WASHINGTON HEADQUARTERS SERVICES (WHS),PENTAGON FORCE PROTECTION AGENCY,VA,AWARD,DO,FIRM FIXED PRICE,SERVICES,NOT A BUNDLED REQUIREMENT,NONE,TRANSACTION DOES NOT USE GFE/GFP,NOT APPLICABLE,Electronic & Communication Services,"INSTALLATION OF EQUIPMENT- ALARM, SIGNAL, AND ...",Administrative and Support and Waste Managemen...,SECURITY SYSTEMS SERVICES (EXCEPT LOCKSMITHS),1,4897.96
3,DEFENSE LOGISTICS AGENCY,DLA DISPOSTION SERVICE - EBS,DEFENSE LOGISTICS AGENCY,DLA DISPOSTION SERVICE - EBS,WI,AWARD,DO,FIRM FIXED PRICE,SERVICES,NOT A BUNDLED REQUIREMENT,NONE,TRANSACTION DOES NOT USE GFE/GFP,NOT APPLICABLE,Facility Related Services,HOUSEKEEPING- WASTE TREATMENT/STORAGE,Administrative and Support and Waste Managemen...,HAZARDOUS WASTE TREATMENT AND DISPOSAL,4,5618.14
4,DEFENSE INFORMATION SYSTEMS AGENCY (DISA),DITCO-SCOTT,DEPT OF THE ARMY,W4NH NETCOM HHC,TX,AWARD,BPA CALL,FIRM FIXED PRICE,SERVICES,NOT A BUNDLED REQUIREMENT,NONE,TRANSACTION DOES NOT USE GFE/GFP,NOT APPLICABLE,Electronic & Communication Services,IT AND TELECOM- TELECOMMUNICATIONS AND TRANSMI...,InformationT,SATELLITE TELECOMMUNICATIONS,1,163.92
7,USTRANSCOM,USTRANSCOM-AQ,DEPT OF THE AIR FORCE,HQ AMC TE,KS,AWARD,DO,FIRM FIXED PRICE,SERVICES,NOT A BUNDLED REQUIREMENT,NONE,TRANSACTION DOES NOT USE GFE/GFP,NOT APPLICABLE,Transportation Services,TRANSPORTATION/TRAVEL/RELOCATION- TRAVEL/LODGI...,Air TransportationT,NONSCHEDULED CHARTERED PASSENGER AIR TRANSPORT...,1,220000.00


In [16]:
df2.describe()

,set_aside_number,contract_value
count,27246.000000,2.724600e+04
mean,1.803274,4.596819e+05
std,1.217351,6.394049e+06
min,1.000000,1.000000e-02
25%,1.000000,2.400000e+03
50%,1.000000,9.800000e+03
75%,2.000000,7.646818e+04
max,12.000000,6.993686e+08


In [17]:
df3 = pd.get_dummies(df2)

In [18]:
X = df3.drop(['set_aside_number'], axis=1)

In [19]:
X.head()

,contract_value,awarding_sub_agency_name_DEFENSE ADVANCED RESEARCH PROJECTS AGENCY (DARPA),awarding_sub_agency_name_DEFENSE COMMISSARY AGENCY (DECA),awarding_sub_agency_name_DEFENSE CONTRACT MANAGEMENT AGENCY (DCMA),awarding_sub_agency_name_DEFENSE COUNTERINTELLIGENCE AND SECURITY AGENCY,awarding_sub_agency_name_DEFENSE FINANCE AND ACCOUNTING SERVICE (DFAS),awarding_sub_agency_name_DEFENSE HEALTH AGENCY (DHA),awarding_sub_agency_name_DEFENSE HUMAN RESOURCES ACTIVITY,awarding_sub_agency_name_DEFENSE INFORMATION SYSTEMS AGENCY (DISA),awarding_sub_agency_name_DEFENSE LOGISTICS AGENCY,awarding_sub_agency_name_DEFENSE MEDIA ACTIVITY (DMA),awarding_sub_agency_name_DEFENSE MICROELECTRONICS ACTIVITY (DMEA),awarding_sub_agency_name_DEFENSE SECURITY COOPERATION AGENCY,awarding_sub_agency_name_DEFENSE THREAT REDUCTION AGENCY (DTRA),awarding_sub_agency_name_DEPT OF DEFENSE EDUCATION ACTIVITY (DODEA),awarding_sub_agency_name_JOINT IMPROVISED EXPLOSIVE DEVICE DEFEAT ORGANIZATION (JIEDDO),awarding_sub_agency_name_JOINT IMPROVISED EXPLOSIVE DEVICE DEFEAT ORGANIZATION (JIEDDO),awarding_sub_agency_name_MISSILE DEFENSE AGENCY (MDA),awarding_sub_agency_name_U.S. SPECIAL OPERATIONS COMMAND (USSOCOM),awarding_sub_agency_name_UNIFORMED SERVICES UNIVERSITY OF THE HEALTH SCIENCES (USUHS),awarding_sub_agency_name_USTRANSCOM,awarding_sub_agency_name_WASHINGTON HEADQUARTERS SERVICES (WHS),awarding_office_name_CONTRACTING OPERATIONS DIVISION,awarding_office_name_DCMA AIMO - ST AUGUSTINE,awarding_office_name_DCMA AIMO ST AUGUSTINE,awarding_office_name_DCMA APO KELLY SAN ANTONIO,awarding_office_name_DCMA BOEING,awarding_office_name_DCSO COLUMBUS,awarding_office_name_DCSO FORT BELVOIR,awarding_office_name_DCSO NEW CUMBERLAND,awarding_office_name_DCSO PHILADELPHIA,awarding_office_name_DCSO RICHMOND,awarding_office_name_DCSO-NEW CUMBERLAND,awarding_office_name_DCSO-P NEW CUMBERLAND,awarding_office_name_DCSO-P/HARRISBURG,awarding_office_name_DDESS AREA SERVICE CENTER,awarding_office_name_DEF ADVANCED RESEARCH PROJECTS AGCY,awarding_office_name_DEFENSE CI AND SECURITY AGENCY,awarding_office_name_DEFENSE COMMISSARY AGENCY,awarding_office_name_DEFENSE CONTRACT MANAGEMENT AGENCY,awarding_office_name_DEFENSE CONTRACT MANAGMENT OFFICE,awarding_office_name_DEFENSE FINANCE AND ACCOUNTING SVC,awarding_office_name_DEFENSE HEALTH AGENCY,awarding_office_name_DEFENSE HEALTH AGENCY - AURORA,awarding_office_name_DEFENSE HEALTH AGENCY - FALLS CHURCH,awarding_office_name_DEFENSE HUMAN RESOURCES ACTIVITY,awarding_office_name_DEFENSE INFORMATION SYSTEMS AGENCY,awarding_office_name_DEFENSE INFORMATION SYSTEMS ANGENCY,awarding_office_name_DEFENSE MICROELECTRONICS ACTIVITY,awarding_office_name_DEFENSE SECURITY COOPERATION AGENCY,awarding_office_name_DEFENSE THREAT REDUCTION AGENCY,awarding_office_name_DES DSCP CONTRACTING SERVICES OFC,awarding_office_name_DISA/DITCO EUROPE,awarding_office_name_DITCO-PAC,awarding_office_name_DITCO-SCOTT,awarding_office_name_DLA DOCUMENT SERVICES,awarding_office_name_DLA ENERGY,"awarding_office_name_DLA AVIATION AT WARNER ROBINS, GA",awarding_office_name_DLA CONTRACTING SERVICES OFFICE,awarding_office_name_DLA CONTRACTING SVCS OFF COLUMBUS,awarding_office_name_DLA DISPOSITION SERVICES - EBS,awarding_office_name_DLA DISPOSITION SERVICES J-761,awarding_office_name_DLA DISPOSTION SERVICE - EBS,awarding_office_name_DLA DISPOSTION SERVICES - PACIFIC,awarding_office_name_DLA DISTRIBUTION,awarding_office_name_DLA ENERGY,awarding_office_name_DLA ENERGY AEROSPACE ENRGY-DLAE-M,awarding_office_name_DLA MARITIME - NORFOLK,awarding_office_name_DLA MARITIME - PUGET SOUND,awarding_office_name_DLA STRATEGIC MATERIALS,awarding_office_name_DLA TROOP SUPPORT,awarding_office_name_DLA TROOP SUPPORT EUR&AF,awarding_office_name_DMA RIVERSIDE,awarding_office_name_DOD EDUCATION ACTIVITY,awarding_office_name_DODDS EUROPE DIRECTOR'S OFFICE,awarding_office_name_DODDS PACIFIC DIRECTOR'S OFFICE,awarding_office_name_HQ DEFENSE MEDIA ACTIVITY,awarding_office_name_HQ USASOC,awarding_

In [20]:
y = df3['set_aside_number']

In [21]:
y

0        1
1        1
3        4
4        1
7        1
        ..
35350    5
35352    1
35355    1
35364    2
35367    1
Name: set_aside_number, Length: 27246, dtype: int64

In [22]:
df3.head()

,set_aside_number,contract_value,awarding_sub_agency_name_DEFENSE ADVANCED RESEARCH PROJECTS AGENCY (DARPA),awarding_sub_agency_name_DEFENSE COMMISSARY AGENCY (DECA),awarding_sub_agency_name_DEFENSE CONTRACT MANAGEMENT AGENCY (DCMA),awarding_sub_agency_name_DEFENSE COUNTERINTELLIGENCE AND SECURITY AGENCY,awarding_sub_agency_name_DEFENSE FINANCE AND ACCOUNTING SERVICE (DFAS),awarding_sub_agency_name_DEFENSE HEALTH AGENCY (DHA),awarding_sub_agency_name_DEFENSE HUMAN RESOURCES ACTIVITY,awarding_sub_agency_name_DEFENSE INFORMATION SYSTEMS AGENCY (DISA),awarding_sub_agency_name_DEFENSE LOGISTICS AGENCY,awarding_sub_agency_name_DEFENSE MEDIA ACTIVITY (DMA),awarding_sub_agency_name_DEFENSE MICROELECTRONICS ACTIVITY (DMEA),awarding_sub_agency_name_DEFENSE SECURITY COOPERATION AGENCY,awarding_sub_agency_name_DEFENSE THREAT REDUCTION AGENCY (DTRA),awarding_sub_agency_name_DEPT OF DEFENSE EDUCATION ACTIVITY (DODEA),awarding_sub_agency_name_JOINT IMPROVISED EXPLOSIVE DEVICE DEFEAT ORGANIZATION (JIEDDO),awarding_sub_agency_name_JOINT IMPROVISED EXPLOSIVE DEVICE DEFEAT ORGANIZATION (JIEDDO),awarding_sub_agency_name_MISSILE DEFENSE AGENCY (MDA),awarding_sub_agency_name_U.S. SPECIAL OPERATIONS COMMAND (USSOCOM),awarding_sub_agency_name_UNIFORMED SERVICES UNIVERSITY OF THE HEALTH SCIENCES (USUHS),awarding_sub_agency_name_USTRANSCOM,awarding_sub_agency_name_WASHINGTON HEADQUARTERS SERVICES (WHS),awarding_office_name_CONTRACTING OPERATIONS DIVISION,awarding_office_name_DCMA AIMO - ST AUGUSTINE,awarding_office_name_DCMA AIMO ST AUGUSTINE,awarding_office_name_DCMA APO KELLY SAN ANTONIO,awarding_office_name_DCMA BOEING,awarding_office_name_DCSO COLUMBUS,awarding_office_name_DCSO FORT BELVOIR,awarding_office_name_DCSO NEW CUMBERLAND,awarding_office_name_DCSO PHILADELPHIA,awarding_office_name_DCSO RICHMOND,awarding_office_name_DCSO-NEW CUMBERLAND,awarding_office_name_DCSO-P NEW CUMBERLAND,awarding_office_name_DCSO-P/HARRISBURG,awarding_office_name_DDESS AREA SERVICE CENTER,awarding_office_name_DEF ADVANCED RESEARCH PROJECTS AGCY,awarding_office_name_DEFENSE CI AND SECURITY AGENCY,awarding_office_name_DEFENSE COMMISSARY AGENCY,awarding_office_name_DEFENSE CONTRACT MANAGEMENT AGENCY,awarding_office_name_DEFENSE CONTRACT MANAGMENT OFFICE,awarding_office_name_DEFENSE FINANCE AND ACCOUNTING SVC,awarding_office_name_DEFENSE HEALTH AGENCY,awarding_office_name_DEFENSE HEALTH AGENCY - AURORA,awarding_office_name_DEFENSE HEALTH AGENCY - FALLS CHURCH,awarding_office_name_DEFENSE HUMAN RESOURCES ACTIVITY,awarding_office_name_DEFENSE INFORMATION SYSTEMS AGENCY,awarding_office_name_DEFENSE INFORMATION SYSTEMS ANGENCY,awarding_office_name_DEFENSE MICROELECTRONICS ACTIVITY,awarding_office_name_DEFENSE SECURITY COOPERATION AGENCY,awarding_office_name_DEFENSE THREAT REDUCTION AGENCY,awarding_office_name_DES DSCP CONTRACTING SERVICES OFC,awarding_office_name_DISA/DITCO EUROPE,awarding_office_name_DITCO-PAC,awarding_office_name_DITCO-SCOTT,awarding_office_name_DLA DOCUMENT SERVICES,awarding_office_name_DLA ENERGY,"awarding_office_name_DLA AVIATION AT WARNER ROBINS, GA",awarding_office_name_DLA CONTRACTING SERVICES OFFICE,awarding_office_name_DLA CONTRACTING SVCS OFF COLUMBUS,awarding_office_name_DLA DISPOSITION SERVICES - EBS,awarding_office_name_DLA DISPOSITION SERVICES J-761,awarding_office_name_DLA DISPOSTION SERVICE - EBS,awarding_office_name_DLA DISPOSTION SERVICES - PACIFIC,awarding_office_name_DLA DISTRIBUTION,awarding_office_name_DLA ENERGY,awarding_office_name_DLA ENERGY AEROSPACE ENRGY-DLAE-M,awarding_office_name_DLA MARITIME - NORFOLK,awarding_office_name_DLA MARITIME - PUGET SOUND,awarding_office_name_DLA STRATEGIC MATERIALS,awarding_office_name_DLA TROOP SUPPORT,awarding_office_name_DLA TROOP SUPPORT EUR&AF,awarding_office_name_DMA RIVERSIDE,awarding_office_name_DOD EDUCATION ACTIVITY,awarding_office_name_DODDS EUROPE DIRECTOR'S OFFICE,awarding_office_name_DODDS PACIFIC DIRECTOR'S OFFICE,awarding_office_name_HQ DEFENSE MEDIA ACTIVITY,awarding_office_name_HQ

In [23]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.naive_bayes import GaussianNB

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [25]:
model = GaussianNB()

In [26]:
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [27]:
predictions = model.predict(X_test)

In [28]:
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test,predictions))

[[ 113  525 2279    0    0    0    0    0    0    0    0]
 [  16  114 1531    0    0    0    0    0    0    0    0]
 [   0    3  287    0    0    0    0    0    0    0    0]
 [   4   32  210    0    0    0    0    0    0    0    0]
 [   8   80  162    0    0    0    0    0    0    0    0]
 [   0   12   40    0    0    0    0    0    0    0    0]
 [   0    6   19    0    0    0    0    0    0    0    0]
 [   0    0    5    0    0    0    0    0    0    0    0]
 [   0    0    1    0    0    0    0    0    0    0    0]
 [   0    1    1    0    0    0    0    0    0    0    0]
 [   0    0    1    0    0    0    0    0    0    0    0]]
              precision    recall  f1-score   support

           1       0.80      0.04      0.07      2917
           2       0.15      0.07      0.09      1661
           3       0.06      0.99      0.12       290
           4       0.00      0.00      0.00       246
           5       0.00      0.00      0.00       250
           6       0.00      0.00   

C:\Users\Fatih\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
score = cross_val_score(estimator=model, X=X_train, y=y_train, cv=12)

C:\Users\Fatih\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=12.
  % (min_groups, self.n_splits)), Warning)


In [30]:
print('Accuracy ', score.mean())

Accuracy  0.09212439276361424
